In [46]:
import numpy as np
import pandas as pd
import torch
import seaborn as sns
from sklearn.manifold import TSNE
# from transformers import BertTokenizer, BertModel, BertForMaskedLM
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('./data.csv')

In [42]:
df

,book,words
0,./data/cook_book_one.txt,project gutenberg's the whitehouse cookbook by...
1,./data/cook_book_three.txt,the project gutenberg ebook of new royal cook ...
2,./data/gothic_novel_four.txt,the project gutenberg ebook of the works of ed...
3,./data/gothic_novel_six.txt,the project gutenberg ebook of northanger abbe...
4,./data/gothic_novel_two.txt,project gutenberg’s the complete works of will...
5,./data/gothic_novel_three.txt,the project gutenberg ebook of dracula by bram...
6,./data/cook_book_four.txt,the project gutenberg ebook of the italian coo...
7,./data/gothic_novel_ten.txt,the project gutenberg ebook of the castle of o...
8,./data/gothic_novel_eight.txt,the project gutenberg ebook of the vampyre a t...
9,./data/gothic_novel_nine.txt,the project gutenberg ebook of the masque of t...


In [9]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 391/391 [00:00<00:00, 137kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3.95k/3.95k [00:00<00:00, 1.41MB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.00/2.00 [00:00<00:00, 1.08kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 229/229 [00:00<00:00, 131kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 438M/438M

In [10]:
document_embeddings = sbert_model.encode(df['words'])

In [52]:
df['embedding'] = document_embeddings

ValueError: Length of values (768) does not match length of index (22)

ValueError: Expected a 1D array, got an array with shape (22, 768)

In [44]:
cosine_similarity(document_embeddings[10].reshape(1, -1), document_embeddings[21].reshape(1, -1))

array([[0.84842074]], dtype=float32)

In [47]:
m = TSNE(learning_rate=100)

In [49]:
tsne_features = m.fit(document_embeddings.reshape(1,-1))

/Users/gael/git-repos/notebooks/book_genre_detection/env/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(


ValueError: Found array with 1 sample(s) (shape=(1, 16896)) while a minimum of 2 is required.

In [ ]:
tsne_features[1:4,:]